# Static Plots

- [pandas.DataFrame.plot — pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html)
- [Matplotlib — Visualization with Python](https://matplotlib.org/)
- [seaborn: statistical data visualization — seaborn 0.11.2 documentation](https://seaborn.pydata.org/)
- [MtkN1/pybotters: An advanced api client for python botters.](https://github.com/MtkN1/pybotters)


## 約定データ

- 時系列の折れ線グラフ
- 価格帯別出来高
- ボリュームの(棒)グラフを下につける
- 指標は代表的なやつを1つ紹介する(SMA, RSIなど)

In [4]:
import os
import sys
import datetime
import requests


In [1]:
def download_zip(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, "wb") as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [3]:
def get_data_url(url):
    r = requests.get(url)
    if r.status_code == 200:
        if r.json()["urls"]:
            return r.json()["urls"][0]["url"]
    return None
def trade_link(endpoint, exchange, instrument, starttime):
    """
    starttime: YYYY-mm-dd
    """
    base_url = f"/trade/{exchange}/{instrument}?startTime={starttime}"
    return endpoint + base_url


def gz_path(exchange, instrument, date, save_dir):
    return os.path.join(f"{save_dir}", f"{exchange}_{instrument}_{date}.gz")


def get_execution_gz(endpoint, exchange, instrument, date, save_dir, download_anyway):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    gzpath = gz_path(exchange, instrument, date, save_dir)
    if os.path.exists(gzpath) and not download_anyway:
        print(
            f"{gzpath} exits. Since download_anyway option is False, Will not Download"
        )
        return

    cryptochassis_trade_api_url = trade_link(endpoint, exchange, instrument, date)
    gz_url = get_data_url(cryptochassis_trade_api_url)

    print(f"DOWNLOADING {endpoint}, {exchange}, {instrument}, {date} TO {gzpath}")
    download_zip(gz_url, gzpath)

In [5]:
endpoint = "https://api.cryptochassis.com/v1"
exchange = "binance"
instrument = "btc-eur"
today = datetime.datetime.utcnow()
date = (today - datetime.timedelta(days=2)).strftime("%Y-%m-%d")
# save_dir = "./data/binance/gz"
save_dir = "/tmp/binance/gz"
download_anyway = True

get_execution_gz(endpoint, exchange, instrument, date, save_dir, download_anyway)




DOWNLOADING https://api.cryptochassis.com/v1, binance, btc-eur, 2022-04-02 TO /tmp/binance/gz/binance_btc-eur_2022-04-02.gz
